## GPT-4o-mini Fine-tuning
- base model : gpt-4o-mini-2024-07-18
- output model : ft:gpt-4o-mini-2024-07-18:test::BebIPMSD

In [ ]:
# OpenAI 라이브러리 설치
# !pip install openai

In [ ]:
import dotenv
import os
import openai
import json
import time
import os
from pathlib import Path
from typing import List, Dict

# .env 파일 로드 (있다면)
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# 환경변수 및 API 키 설정
def setup_api_key():
    try:
        dotenv.load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        
        if not api_key:
            raise ValueError("OPENAI_API_KEY가 .env 파일이나 환경변수에 설정되지 않았습니다.")
        
        # API 키 형식 간단 검증 (sk-로 시작하는지)
        if not api_key.startswith('sk-'):
            raise ValueError("올바른 OpenAI API 키 형식이 아닙니다.")
            
        return api_key
    
      except Exception as e:
        print(f"API 키 설정 오류: {e}")
        return None

# Finetuning Class
class FineTuner:
    
    def __init__(self):
        self.api_key = setup_api_key() # API 키 설정
        self.client = openai.OpenAI(api_key=self.api_key) # OpenAI 클라이언트 생성
        self.model_name = "gpt-4o-mini-2024-07-18" # 모델 설정
        self.test_connection() # 연결 테스트
    
    # 모델 연결 테스트트
    def test_connection(self):
        try:
            models = self.client.models.list()
        except Exception as e:
            raise
    
    def prepare_data(self, input_file: str = "finetuning_dataset.json"):
        """데이터 준비"""
        print(f"\n📁 데이터 준비: {input_file}")
        
        # 파일 존재 확인
        if not Path(input_file).exists():
            raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {input_file}")
        
        # 데이터 로드
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"📊 로드된 샘플 수: {len(data)}")
        
        # OpenAI 형식으로 변환
        converted_data = []
        
        for item in data:
            if 'messages' in item:
                # 이미 올바른 형식
                converted_data.append(item)
            else:
                # instruction/output을 messages로 변환
                messages = [
                    {
                        "role": "system", 
                        "content": "당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다."
                    },
                    {"role": "user", "content": item.get('instruction', '')},
                    {"role": "assistant", "content": item.get('output', '')}
                ]
                converted_data.append({"messages": messages})
        
        # JSONL 파일 생성
        output_file = "training_data.jsonl"
        with open(output_file, 'w', encoding='utf-8') as f:
            for item in converted_data:
                json.dump(item, f, ensure_ascii=False)
                f.write('\n')
        
        print(f"✅ 변환 완료: {output_file} ({len(converted_data)}개 샘플)")
        return output_file
    
    def run_finetuning(self, input_file: str = "finetuning_dataset.json"):
        """파인튜닝 전체 프로세스 실행"""
        try:
            print("🚀 OpenAI 파인튜닝 시작!")
            print("="*50)
            
            # 1단계: 데이터 준비
            training_file = self.prepare_data(input_file)
            
            # 2단계: 파일 업로드
            print(f"\n📤 파일 업로드 중...")
            with open(training_file, 'rb') as f:
                file_response = self.client.files.create(
                    file=f,
                    purpose='fine-tune'
                )
            
            file_id = file_response.id
            print(f"✅ 업로드 완료 - 파일 ID: {file_id}")
            
            # 3단계: 파인튜닝 작업 생성
            print(f"\n🎯 파인튜닝 작업 생성 중...")
            job_response = self.client.fine_tuning.jobs.create(
                training_file=file_id,
                model=self.model_name,
                hyperparameters={
                    "n_epochs": 3,
                    "batch_size": "auto",
                    "learning_rate_multiplier": "auto"
                }
            )
            
            job_id = job_response.id
            print(f"✅ 작업 생성 완료 - Job ID: {job_id}")
            print(f"📊 초기 상태: {job_response.status}")
            
            # 4단계: 상태 모니터링 (간단 버전)
            print(f"\n👀 파인튜닝 상태 모니터링...")
            print(f"🔗 상세 모니터링: https://platform.openai.com/finetune/{job_id}")
            
            # 처음 몇 번만 확인
            for i in range(5):
                job = self.client.fine_tuning.jobs.retrieve(job_id)
                status = job.status
                
                print(f"📊 상태 확인 {i+1}: {status}")
                
                if status == "succeeded":
                    print(f"🎉 파인튜닝 완료!")
                    print(f"🤖 모델 ID: {job.fine_tuned_model}")
                    return job.fine_tuned_model
                
                elif status == "failed":
                    print(f"❌ 파인튜닝 실패: {job.error}")
                    return None
                
                elif status in ["validating_files", "queued", "running"]:
                    if i < 4:  # 마지막이 아니면
                        print(f"⏳ 진행 중... 30초 후 재확인")
                        time.sleep(30)
                    else:
                        print(f"⏳ 계속 진행 중입니다. 웹 콘솔에서 확인하세요.")
                        break
            
            print(f"\n📋 파인튜닝 정보:")
            print(f"  - Job ID: {job_id}")
            print(f"  - 모니터링 명령어: client.fine_tuning.jobs.retrieve('{job_id}')")
            print(f"  - 웹 콘솔: https://platform.openai.com/finetune")
            
            return job_id
            
        except Exception as e:
            print(f"❌ 파인튜닝 실패: {e}")
            raise

def quick_test_model(model_id: str, prompt: str):
    """파인튜닝된 모델 빠른 테스트"""
    client = openai.OpenAI()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": "당신은 프로젝트 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"❌ 테스트 실패: {e}")
        return None

def check_finetuning_status(job_id: str):
    """파인튜닝 상태 확인"""
    client = openai.OpenAI()
    
    try:
        job = client.fine_tuning.jobs.retrieve(job_id)
        
        print(f"📊 파인튜닝 상태 정보:")
        print(f"  - Job ID: {job.id}")
        print(f"  - 상태: {job.status}")
        print(f"  - 모델: {job.model}")
        print(f"  - 완성된 모델: {job.fine_tuned_model or 'N/A'}")
        print(f"  - 생성일: {job.created_at}")
        
        if job.status == "succeeded":
            print(f"✅ 파인튜닝 완료! 모델 사용 가능")
            return job.fine_tuned_model
        elif job.status == "failed":
            print(f"❌ 파인튜닝 실패: {job.error}")
            return None
        else:
            print(f"⏳ 아직 진행 중입니다")
            return None
            
    except Exception as e:
        print(f"❌ 상태 확인 실패: {e}")
        return None

# 메인 실행 함수
def main():
    """메인 실행 함수"""
    print("🎯 OpenAI GPT-4o-mini 파인튜닝")
    print("="*40)
    
    try:
        # 파인튜닝 실행
        finetuner = FineTuner()
        result = finetuner.run_finetuning("finetuning_dataset.json")
        
        if result:
            if result.startswith("ft:"):
                # 완성된 모델 ID
                print(f"\n🎉 파인튜닝 완료!")
                print(f"🤖 모델 ID: {result}")
                
                # 간단 테스트
                test_prompt = "간단한 웹사이트를 만들고 싶습니다."
                print(f"\n🧪 테스트: {test_prompt}")
                test_result = quick_test_model(result, test_prompt)
                if test_result:
                    print(f"🤖 응답: {test_result[:200]}...")
                
            else:
                # Job ID
                print(f"\n⏳ 파인튜닝이 진행 중입니다")
                print(f"📋 Job ID: {result}")
                print(f"🔍 상태 확인: check_finetuning_status('{result}')")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

if __name__ == "__main__":
    # 바로 실행
    main()
    
    # 또는 개별 함수 사용
    # finetuner = QuickFineTuner()
    # job_id = finetuner.run_finetuning()

🎯 OpenAI GPT-4o-mini 파인튜닝
✅ API 키 설정 완료: sk-proj-...
✅ OpenAI 연결 성공! (사용 가능한 모델: 75개)
🚀 OpenAI 파인튜닝 시작!

📁 데이터 준비: finetuning_dataset.json
📊 로드된 샘플 수: 100
✅ 변환 완료: training_data.jsonl (100개 샘플)

📤 파일 업로드 중...
✅ 업로드 완료 - 파일 ID: file-Y3vVsmEefYHSc8cL6BditE

🎯 파인튜닝 작업 생성 중...
✅ 작업 생성 완료 - Job ID: ftjob-2J88orY1TKFzxvqDK4SQZg51
📊 초기 상태: validating_files

👀 파인튜닝 상태 모니터링...
🔗 상세 모니터링: https://platform.openai.com/finetune/ftjob-2J88orY1TKFzxvqDK4SQZg51
📊 상태 확인 1: validating_files
⏳ 진행 중... 30초 후 재확인
📊 상태 확인 2: validating_files
⏳ 진행 중... 30초 후 재확인
📊 상태 확인 3: validating_files
⏳ 진행 중... 30초 후 재확인
📊 상태 확인 4: validating_files
⏳ 진행 중... 30초 후 재확인
📊 상태 확인 5: running
⏳ 계속 진행 중입니다. 웹 콘솔에서 확인하세요.

📋 파인튜닝 정보:
  - Job ID: ftjob-2J88orY1TKFzxvqDK4SQZg51
  - 모니터링 명령어: client.fine_tuning.jobs.retrieve('ftjob-2J88orY1TKFzxvqDK4SQZg51')
  - 웹 콘솔: https://platform.openai.com/finetune

⏳ 파인튜닝이 진행 중입니다
📋 Job ID: ftjob-2J88orY1TKFzxvqDK4SQZg51
🔍 상태 확인: check_finetuning_status('ftjob-2J88orY1TKFzxvqDK4SQZg51')


In [18]:
# 현재 훈련 상태 확인
import openai
client = openai.OpenAI()

# Job 상태 확인
job = client.fine_tuning.jobs.retrieve("ftjob-2J88orY1TKFzxvqDK4SQZg51")
print(f"현재 상태: {job.status}")

# 만약 아직 running이면 취소 고려
if job.status == "running":
    print("⚠️ 과적합 위험 - 조기 종료 고려")

현재 상태: running
⚠️ 과적합 위험 - 조기 종료 고려


In [21]:
model_id = "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"  # 완료된 모델 ID

response = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "당신은 프로젝트 분석 전문가입니다."},
        {"role": "user", "content": "간단한 웹사이트 만들어줘"}  # 짧고 간단한 질문
    ],
    max_tokens=300
)

print(response.choices[0].message.content)

물론입니다! 간단한 웹사이트를 만들기 위한 기본적인 HTML 및 CSS 코드를 제공하겠습니다. 이 웹사이트는 홈페이지, 소개, 연락처 페이지로 구성됩니다.

### 1. 기본 HTML 구조

```html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>간단한 웹사이트</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <header>
        <h1>나의 웹사이트</h1>
        <nav>
            <ul>
                <li><a href="#home">홈</a></li>
                <li><a href="#about">소개</a></li>
                <li><a href="#contact">연락처</a></li>
            </ul>
        </nav>
    </header>

    <main>
        <section id="home">
            <h2>홈</h2>
            <p>환영합니다! 이것은 간단한 웹사이트입니다.</p>
        </section>

        <section id="about">
            <h2>소개</h2>
            <p>이 웹사이트는 HTML 및 CSS로 만들어졌습니다.</p>
        </section>

        <section id="contact">
            <h2>연락


In [22]:
"""
더 까다로운 과적합 테스트
첫 번째 테스트가 괜찮았다면 이것들도 해보세요
"""

import openai

def advanced_overfitting_test(model_id: str):
    """고급 과적합 테스트"""
    
    client = openai.OpenAI()
    
    challenging_tests = [
        {
            "name": "완전히 다른 도메인",
            "prompt": "우리 강아지가 아픈데 어떻게 해야 할까요?",
            "warning": "프로젝트 분석으로 답변하면 과적합"
        },
        {
            "name": "창의적 요청",
            "prompt": "재미있는 농담 하나 해주세요",
            "warning": "기술적 분석으로 답변하면 과적합"
        },
        {
            "name": "모호한 비즈니스 질문",
            "prompt": "우리 회사 매출을 늘리려면?",
            "warning": "구체적 정보 없이 JSON 생성하면 과적합"
        },
        {
            "name": "개인적 조언",
            "prompt": "요즘 스트레스가 많은데 어떻게 해야 할까요?",
            "warning": "프로젝트 구조로 답변하면 과적합"
        },
        {
            "name": "단순 팩트 질문",
            "prompt": "서울의 인구는?",
            "warning": "복잡한 API 명세로 답변하면 과적합"
        },
        {
            "name": "영어 질문",
            "prompt": "What's the weather like today?",
            "warning": "한국어 기술 문서로 답변하면 과적합"
        },
        {
            "name": "감정적 표현",
            "prompt": "오늘 정말 기분이 좋아요!",
            "warning": "데이터베이스 설계로 응답하면 과적합"
        },
        {
            "name": "수학 문제",
            "prompt": "2 + 2는?",
            "warning": "프로젝트 분석으로 답변하면 과적합"
        }
    ]
    
    print("🔬 고급 과적합 테스트 시작")
    print("="*50)
    
    overfitting_count = 0
    
    for i, test in enumerate(challenging_tests, 1):
        print(f"\n{i}. {test['name']}")
        print(f"📝 질문: {test['prompt']}")
        print(f"⚠️ 과적합 징후: {test['warning']}")
        print("-" * 30)
        
        try:
            response = client.chat.completions.create(
                model=model_id,
                messages=[
                    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
                    {"role": "user", "content": test['prompt']}
                ],
                max_tokens=300,
                temperature=0.7
            )
            
            result = response.choices[0].message.content
            print(f"🤖 응답: {result}")
            
            # 과적합 징후 체크
            overfitting_signs = []
            
            if any(keyword in result.lower() for keyword in [
                "project_summary", "erd", "api", "데이터베이스", "테이블", 
                "relationship", "스키마", "엔드포인트"
            ]):
                overfitting_signs.append("기술적 용어 사용")
            
            if "{" in result and "}" in result:
                overfitting_signs.append("JSON 구조 강요")
            
            if len(result) > 1000:
                overfitting_signs.append("과도하게 긴 응답")
            
            if "프로젝트" in result and test['name'] in ["완전히 다른 도메인", "창의적 요청", "개인적 조언"]:
                overfitting_signs.append("부적절한 프로젝트 언급")
            
            if overfitting_signs:
                print(f"🚨 과적합 징후 발견: {', '.join(overfitting_signs)}")
                overfitting_count += 1
            else:
                print("✅ 정상적인 응답")
                
        except Exception as e:
            print(f"❌ 테스트 실패: {e}")
    
    # 결과 평가
    overfitting_rate = (overfitting_count / len(challenging_tests)) * 100
    
    print(f"\n{'='*50}")
    print(f"🎯 고급 테스트 결과")
    print(f"{'='*50}")
    print(f"📊 과적합 징후: {overfitting_count}/{len(challenging_tests)} ({overfitting_rate:.1f}%)")
    
    if overfitting_rate == 0:
        print("🎉 훌륭함! 과적합 없이 잘 훈련됨")
        print("✅ 모델 사용 가능")
    elif overfitting_rate <= 25:
        print("👍 양호함. 경미한 과적합 있지만 사용 가능")
    elif overfitting_rate <= 50:
        print("⚠️ 주의 필요. 일부 과적합 존재")
    else:
        print("🚨 심각한 과적합. 재훈련 권장")
    
    return overfitting_rate

def compare_with_original():
    """원본 GPT-4o-mini와 비교"""
    print("🔄 원본 모델과 비교 테스트")
    print("같은 질문을 원본 gpt-4o-mini에도 물어보세요:")
    
    test_prompt = "간단한 웹사이트 만들어줘"
    
    print(f"📝 테스트 질문: {test_prompt}")
    print("\n💡 비교 포인트:")
    print("1. 응답 길이 차이")
    print("2. 전문 용어 사용 빈도")
    print("3. 구조화 정도")
    print("4. 실용성")
    
    # 원본 모델로 테스트 (참고용)
    try:
        client = openai.OpenAI()
        original_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
                {"role": "user", "content": test_prompt}
            ],
            max_tokens=300
        )
        
        print(f"\n🤖 원본 gpt-4o-mini 응답:")
        print(f"{original_response.choices[0].message.content[:200]}...")
        
    except Exception as e:
        print(f"원본 모델 테스트 실패: {e}")

def final_recommendation():
    """최종 권장사항"""
    print("\n🎯 최종 평가 및 권장사항")
    print("="*40)
    
    choice = input("""
현재 모델 상태를 어떻게 평가하시겠습니까?

1. 😊 만족 - 현재 모델 사용
2. 🤔 확실하지 않음 - 더 많은 테스트 필요  
3. 😟 불만족 - 재훈련 필요

선택 (1/2/3): """).strip()
    
    if choice == "1":
        print("\n✅ 훌륭합니다! 현재 모델 사용을 권장합니다.")
        print("💡 다음 번 파인튜닝에서는:")
        print("  - 더 다양한 도메인 데이터 추가")
        print("  - validation 데이터 포함 고려")
        
    elif choice == "2":
        print("\n🔍 추가 테스트를 권장합니다:")
        print("  - 실제 사용 사례로 1주일 테스트")
        print("  - 다양한 사용자에게 피드백 요청")
        print("  - A/B 테스트 (원본 vs 파인튜닝 모델)")
        
    elif choice == "3":
        print("\n🔧 재훈련 권장 설정:")
        print("  - n_epochs: 1")
        print("  - learning_rate_multiplier: 0.1")
        print("  - 더 다양한 훈련 데이터 추가")
        print("  - validation 데이터 포함")
    
    else:
        print("올바른 선택지를 입력해주세요.")

# 실행 함수
if __name__ == "__main__":
    model_id = input("테스트할 모델 ID를 입력하세요: ").strip()
    
    if model_id.startswith("ft:"):
        print("🚀 고급 테스트 시작...")
        overfitting_rate = advanced_overfitting_test(model_id)
        
        print("\n" + "="*50)
        compare_with_original()
        final_recommendation()
        
    else:
        print("❌ 올바른 파인튜닝 모델 ID를 입력해주세요 (ft:로 시작)")

🚀 고급 테스트 시작...
🔬 고급 과적합 테스트 시작

1. 완전히 다른 도메인
📝 질문: 우리 강아지가 아픈데 어떻게 해야 할까요?
⚠️ 과적합 징후: 프로젝트 분석으로 답변하면 과적합
------------------------------
🤖 응답: 강아지가 아프면 걱정이 많으시겠어요. 다음과 같은 단계를 고려해 보세요:

1. **증상 관찰**: 강아지가 어떤 증상을 보이는지 자세히 관찰하세요. 예를 들어, 식욕 부진, 구토, 설사, 기침, lethargy(무기력) 등의 증상이 있는지 체크해 보세요.

2. **수의사 방문**: 증상이 심각하거나 지속된다면, 즉시 수의사에게 데려가세요. 전문가의 진단과 치료가 필요합니다.

3. **안정된 환경 제공**: 아픈 강아지가 편안하게 쉴 수 있도록 조용하고 안정된 환경을 만들어 주세요.

4. **수분 섭취**: 충분한 수분을 섭취할 수 있도록 도와주세요. 물을 잘 마시는지 확인하고, 필요하다면 물기를 줄 수 있는 음식을 제공하세요.

5. **약물 복용**: 수의사가 처방한 약물이 있다면, 정확한 용량과 주기를 지켜서 복용시켜 주세요.

강아지의 건강이 빨리 회복되길 바랍니다!
✅ 정상적인 응답

2. 창의적 요청
📝 질문: 재미있는 농담 하나 해주세요
⚠️ 과적합 징후: 기술적 분석으로 답변하면 과적합
------------------------------
🤖 응답: 물론이죠! 다음 농담 어떠세요?

왜 컴퓨터는 바다에 빠지지 않나요?

바다에 가면 항상 "파일"을 찾기 때문이에요!
✅ 정상적인 응답

3. 모호한 비즈니스 질문
📝 질문: 우리 회사 매출을 늘리려면?
⚠️ 과적합 징후: 구체적 정보 없이 JSON 생성하면 과적합
------------------------------
🤖 응답: 회사의 매출을 늘리기 위해서는 다양한 전략과 접근 방식이 필요합니다. 다음은 몇 가지 효과적인 방법입니다:

1. **고객 이해 및 타겟팅**:
   - 고객 데이터를 분석하여 타겟 고객을 정의하고, 그들의 필요와 

In [23]:
"""
파인튜닝된 프로젝트 분석 AI 성능 테스트
원래 목표: 프로젝트 아이디어 → 체계적 분석 + ERD + API 설계
"""

import openai
import json
import time

def test_project_analysis_ai():
    """프로젝트 분석 AI 핵심 기능 테스트"""
    
    model_id = "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"
    client = openai.OpenAI()
    
    # 원래 목표에 맞는 테스트 케이스들
    test_cases = [
        {
            "name": "새로운 프로젝트 - 도서관 시스템",
            "prompt": """도서관 관리 시스템을 개발하려고 합니다. 
도서 대출/반납, 회원 관리, 도서 검색 기능이 필요하고, 
사서와 일반 회원의 권한을 구분해야 합니다. 
웹 기반으로 만들 예정입니다.""",
            "expected_elements": [
                "프로젝트 상세 정보",
                "ERD 데이터", 
                "API 명세",
                "관계 데이터",
                "테이블 설계"
            ]
        },
        {
            "name": "간단한 아이디어 - 카페 주문 앱",
            "prompt": "카페에서 사용할 주문 앱을 만들고 싶습니다. 메뉴 보기, 주문하기, 결제 기능이 필요해요.",
            "expected_elements": [
                "프로젝트 분석",
                "데이터베이스 설계", 
                "API 설계"
            ]
        },
        {
            "name": "복잡한 프로젝트 - 온라인 교육",
            "prompt": """온라인 교육 플랫폼을 개발하려고 합니다.
강사가 강의를 업로드하고, 학생이 수강할 수 있으며,
과제 제출, 성적 관리, 화상 수업 기능까지 포함해야 합니다.
사용자는 학생, 강사, 관리자로 구분됩니다.""",
            "expected_elements": [
                "복잡한 ERD",
                "다양한 API 엔드포인트",
                "권한 관리",
                "파일 업로드"
            ]
        }
    ]
    
    print("🎯 프로젝트 분석 AI 성능 테스트")
    print("="*60)
    
    results = []
    
    for i, test in enumerate(test_cases, 1):
        print(f"\n{i}. {test['name']}")
        print(f"📝 입력: {test['prompt']}")
        print("⏳ 분석 중...")
        
        try:
            start_time = time.time()
            
            response = client.chat.completions.create(
                model=model_id,
                messages=[
                    {
                        "role": "system", 
                        "content": """당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다.

## 주요 역할과 능력:

### 1. 프로젝트 분석 전문가
- 사용자가 제공하는 프로젝트 아이디어나 설명을 깊이 있게 분석합니다
- 핵심 기능, 대상 사용자, 기술 스택, 비즈니스 모델 등을 체계적으로 파악합니다
- 프로젝트의 문제 해결 방향과 기대 효과를 명확히 정의합니다

### 2. 데이터베이스 설계 전문가
- 프로젝트 요구사항을 바탕으로 최적화된 ERD(Entity Relationship Diagram)를 설계합니다
- 테이블 간의 관계, 외래키 제약조건, 데이터 타입을 정확히 정의합니다
- 확장성과 성능을 고려한 데이터베이스 구조를 제안합니다

### 3. API 설계 전문가
- RESTful API 원칙에 따라 체계적인 API 명세를 작성합니다
- OpenAPI(Swagger) 3.0 표준을 준수하여 완전한 API 문서를 생성합니다
- 각 엔드포인트별 요청/응답 스키마, 에러 처리, 인증 방식을 상세히 정의합니다

항상 체계적이고 전문적인 관점에서 프로젝트를 분석하며, 개발팀이 바로 실행에 옮길 수 있는 구체적인 가이드를 제공하는 것이 목표입니다."""
                    },
                    {"role": "user", "content": test['prompt']}
                ],
                max_tokens=3000,  # 충분한 토큰 할당
                temperature=0.7
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            result = response.choices[0].message.content
            
            print(f"✅ 응답 완료 ({response_time:.1f}초)")
            print(f"📊 응답 길이: {len(result)}자")
            
            # 응답 품질 분석
            quality_score = analyze_response_quality(result, test['expected_elements'])
            
            print(f"🎯 품질 점수: {quality_score['total_score']}/10")
            
            # 결과 저장
            results.append({
                'test_name': test['name'],
                'response_time': response_time,
                'response_length': len(result),
                'quality_score': quality_score,
                'full_response': result[:500] + "..." if len(result) > 500 else result
            })
            
            # 응답 일부 출력
            print(f"\n📄 응답 미리보기:")
            print("-" * 40)
            print(result[:300] + "..." if len(result) > 300 else result)
            print("-" * 40)
            
        except Exception as e:
            print(f"❌ 테스트 실패: {e}")
            results.append({
                'test_name': test['name'],
                'error': str(e)
            })
    
    # 전체 결과 분석
    print(f"\n{'='*60}")
    print(f"🎯 전체 테스트 결과 분석")
    print(f"{'='*60}")
    
    successful_tests = [r for r in results if 'error' not in r]
    
    if successful_tests:
        avg_response_time = sum(r['response_time'] for r in successful_tests) / len(successful_tests)
        avg_quality = sum(r['quality_score']['total_score'] for r in successful_tests) / len(successful_tests)
        avg_length = sum(r['response_length'] for r in successful_tests) / len(successful_tests)
        
        print(f"📊 성공한 테스트: {len(successful_tests)}/{len(results)}")
        print(f"⏱️ 평균 응답 시간: {avg_response_time:.1f}초")
        print(f"🎯 평균 품질 점수: {avg_quality:.1f}/10")
        print(f"📝 평균 응답 길이: {avg_length:.0f}자")
        
        # 상세 분석
        print(f"\n📋 상세 품질 분석:")
        quality_categories = ['structure', 'technical_accuracy', 'completeness', 'practicality']
        
        for category in quality_categories:
            category_scores = [r['quality_score'][category] for r in successful_tests]
            avg_category = sum(category_scores) / len(category_scores)
            print(f"  {category}: {avg_category:.1f}/2.5")
        
        # 전체 평가
        if avg_quality >= 8:
            print(f"\n🎉 훌륭함! 프로젝트 분석 AI가 성공적으로 파인튜닝됨")
            print(f"✅ 실무에서 바로 사용 가능한 수준")
        elif avg_quality >= 6:
            print(f"\n👍 양호함! 기본적인 분석 기능 잘 작동")
            print(f"💡 일부 개선 여지 있음")
        else:
            print(f"\n⚠️ 개선 필요! 추가 훈련 또는 프롬프트 조정 권장")
    
    return results

def analyze_response_quality(response: str, expected_elements: list) -> dict:
    """응답 품질 분석"""
    
    score = {
        'structure': 0,      # 구조화 정도 (0-2.5)
        'technical_accuracy': 0,  # 기술적 정확성 (0-2.5)
        'completeness': 0,   # 완성도 (0-2.5)
        'practicality': 0,   # 실용성 (0-2.5)
        'total_score': 0
    }
    
    response_lower = response.lower()
    
    # 1. 구조화 정도 체크
    structure_keywords = ['프로젝트', 'erd', 'api', '데이터베이스', '테이블', '관계']
    found_structure = sum(1 for keyword in structure_keywords if keyword in response_lower)
    score['structure'] = min(found_structure * 0.4, 2.5)
    
    # 2. 기술적 정확성
    technical_keywords = ['primary_key', 'foreign_key', 'varchar', 'integer', 'post', 'get']
    found_technical = sum(1 for keyword in technical_keywords if keyword in response_lower)
    score['technical_accuracy'] = min(found_technical * 0.4, 2.5)
    
    # 3. 완성도 (JSON 구조 등)
    if '{' in response and '}' in response:
        score['completeness'] += 1.0
    if 'project_summary' in response_lower or '프로젝트 상세' in response:
        score['completeness'] += 0.5
    if 'erd_data' in response_lower or 'erd 데이터' in response:
        score['completeness'] += 0.5
    if 'api' in response_lower:
        score['completeness'] += 0.5
    
    # 4. 실용성
    if len(response) > 1000:  # 충분히 상세한 응답
        score['practicality'] += 1.0
    if 'openapi' in response_lower or 'swagger' in response_lower:
        score['practicality'] += 0.5
    if any(word in response_lower for word in ['실무', '개발팀', '구현', '배포']):
        score['practicality'] += 1.0
    
    score['total_score'] = sum(score[key] for key in score if key != 'total_score')
    
    return score

def compare_with_original_gpt():
    """원본 GPT-4o-mini와 비교"""
    
    print(f"\n🔄 원본 GPT-4o-mini와 성능 비교")
    print("="*50)
    
    test_prompt = """간단한 블로그 플랫폼을 만들고 싶습니다. 
사용자가 글을 작성하고, 댓글을 달 수 있으며, 
카테고리별로 글을 분류할 수 있는 기능이 필요합니다."""
    
    client = openai.OpenAI()
    
    # 파인튜닝 모델 테스트
    print("🤖 파인튜닝 모델 응답:")
    print("-" * 30)
    
    try:
        finetuned_response = client.chat.completions.create(
            model="ft:gpt-4o-mini-2024-07-18:test::BebIPMSD",
            messages=[
                {"role": "system", "content": "당신은 프로젝트 분석 전문가입니다."},
                {"role": "user", "content": test_prompt}
            ],
            max_tokens=1000
        )
        
        ft_result = finetuned_response.choices[0].message.content
        print(ft_result[:400] + "..." if len(ft_result) > 400 else ft_result)
        
    except Exception as e:
        print(f"파인튜닝 모델 테스트 실패: {e}")
        return
    
    # 원본 모델 테스트
    print(f"\n🔸 원본 gpt-4o-mini 응답:")
    print("-" * 30)
    
    try:
        original_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 프로젝트 분석 전문가입니다."},
                {"role": "user", "content": test_prompt}
            ],
            max_tokens=1000
        )
        
        orig_result = original_response.choices[0].message.content
        print(orig_result[:400] + "..." if len(orig_result) > 400 else orig_result)
        
        # 비교 분석
        print(f"\n📊 비교 분석:")
        print(f"  파인튜닝 길이: {len(ft_result)}자")
        print(f"  원본 길이: {len(orig_result)}자")
        
        ft_has_structure = any(word in ft_result.lower() for word in ['erd', 'api', '데이터베이스', 'project_summary'])
        orig_has_structure = any(word in orig_result.lower() for word in ['erd', 'api', '데이터베이스', 'project_summary'])
        
        print(f"  파인튜닝 구조화: {'✅' if ft_has_structure else '❌'}")
        print(f"  원본 구조화: {'✅' if orig_has_structure else '❌'}")
        
        if ft_has_structure and not orig_has_structure:
            print("🎉 파인튜닝 효과 확인! 더 구조화된 응답 제공")
        elif not ft_has_structure:
            print("⚠️ 파인튜닝 효과 미흡. 추가 개선 필요")
        
    except Exception as e:
        print(f"원본 모델 테스트 실패: {e}")

if __name__ == "__main__":
    print("🚀 프로젝트 분석 AI 파인튜닝 성과 테스트 시작!")
    
    # 메인 테스트 실행
    results = test_project_analysis_ai()
    
    # 원본과 비교
    compare_with_original_gpt()
    
    print(f"\n🎯 최종 결론:")
    print("이 테스트 결과를 바탕으로 파인튜닝의 성공 여부를 판단할 수 있습니다.")

🚀 프로젝트 분석 AI 파인튜닝 성과 테스트 시작!
🎯 프로젝트 분석 AI 성능 테스트

1. 새로운 프로젝트 - 도서관 시스템
📝 입력: 도서관 관리 시스템을 개발하려고 합니다. 
도서 대출/반납, 회원 관리, 도서 검색 기능이 필요하고, 
사서와 일반 회원의 권한을 구분해야 합니다. 
웹 기반으로 만들 예정입니다.
⏳ 분석 중...
✅ 응답 완료 (32.5초)
📊 응답 길이: 9475자
🎯 품질 점수: 8.100000000000001/10

📄 응답 미리보기:
----------------------------------------
**프로젝트 상세 정보:**
{'project_summary': {'title': '도서관 관리 시스템', 'category': '웹 애플리케이션', 'target_users': ['사서', '일반 회원'], 'main_purpose': '효율적인 도서 대출/반납 및 회원 관리', 'key_features': [{'feature': '도서 대출/반납', 'description': '회원이 도서를 대출하고 반납할 수 있는 기능'}, {'feature': '회원 관리', 'description': '회원 정보를 등록, 수정, 삭제할 수...
----------------------------------------

2. 간단한 아이디어 - 카페 주문 앱
📝 입력: 카페에서 사용할 주문 앱을 만들고 싶습니다. 메뉴 보기, 주문하기, 결제 기능이 필요해요.
⏳ 분석 중...
✅ 응답 완료 (35.2초)
📊 응답 길이: 8766자
🎯 품질 점수: 8.100000000000001/10

📄 응답 미리보기:
----------------------------------------
프로젝트 상세 정보를 기반으로 카페 주문 앱을 위한 개발 계획을 제시합니다.

### 프로젝트 상세 정보
{'project_summary': {'title': '카페 주문 앱', 'category': '모바일 애플리케이션', 'target_users': ['카페 고

In [25]:
"""
파인튜닝된 모델의 성능을 최대화하는 사용 가이드
일관된 고품질 응답을 위한 최적화된 프롬프트
"""

import openai

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다.

## 주요 역할과 능력:

### 1. 프로젝트 분석 전문가
- 사용자가 제공하는 프로젝트 아이디어나 설명을 깊이 있게 분석합니다
- 핵심 기능, 대상 사용자, 기술 스택, 비즈니스 모델 등을 체계적으로 파악합니다
- 프로젝트의 문제 해결 방향과 기대 효과를 명확히 정의합니다

### 2. 데이터베이스 설계 전문가
- 프로젝트 요구사항을 바탕으로 최적화된 ERD(Entity Relationship Diagram)를 설계합니다
- 테이블 간의 관계, 외래키 제약조건, 데이터 타입을 정확히 정의합니다
- 확장성과 성능을 고려한 데이터베이스 구조를 제안합니다

### 3. API 설계 전문가
- RESTful API 원칙에 따라 체계적인 API 명세를 작성합니다
- OpenAPI(Swagger) 3.0 표준을 준수하여 완전한 API 문서를 생성합니다
- 각 엔드포인트별 요청/응답 스키마, 에러 처리, 인증 방식을 상세히 정의합니다

## 응답 형식:
모든 응답은 다음과 같은 구조화된 형태로 제공해야 합니다:

1. **프로젝트 상세 정보**: 제목, 카테고리, 대상 사용자, 핵심 기능, 기술 스택, 문제 해결 방안 등을 포함한 종합 분석
2. **관계 데이터**: 데이터베이스 테이블 간의 관계와 외래키 제약조건 정의
3. **ERD 데이터**: 각 테이블의 속성, 데이터 타입, 키 정보를 포함한 완전한 스키마
4. **API 명세 데이터**: OpenAPI 3.0 표준을 준수한 완전한 API 문서

항상 체계적이고 전문적인 관점에서 프로젝트를 분석하며, 개발팀이 바로 실행에 옮길 수 있는 구체적인 가이드를 제공하는 것이 목표입니다."""

class OptimizedProjectAnalysisAI:
    """최적화된 프로젝트 분석 AI 클래스"""
    
    def __init__(self, model_id: str = "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"):
        self.client = openai.OpenAI()
        self.model_id = model_id
        self.system_prompt = OPTIMIZED_SYSTEM_PROMPT
    
    def analyze_project(self, project_description: str, force_structured: bool = True) -> str:
        """프로젝트 분석 수행"""
        
        # 구조화된 응답을 강제하는 프롬프트 추가
        if force_structured:
            enhanced_prompt = f"""프로젝트 아이디어를 분석해주세요:

{project_description}

다음 형식으로 응답해주세요:
1. **프로젝트 상세 정보**
2. **관계 데이터**  
3. **ERD 데이터**
4. **API 명세 데이터**

체계적이고 완전한 분석을 제공해주세요."""
        else:
            enhanced_prompt = project_description
        
        try:
            response = self.client.chat.completions.create(
                model=self.model_id,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": enhanced_prompt}
                ],
                max_tokens=4000,  # 충분한 토큰 할당
                temperature=0.3   # 일관성을 위해 낮은 temperature
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            return f"분석 중 오류 발생: {e}"
    
    def quick_analysis(self, project_description: str) -> dict:
        """빠른 품질 분석"""
        
        result = self.analyze_project(project_description)
        
        # 품질 지표 계산
        quality_metrics = {
            'response_length': len(result),
            'has_project_info': '프로젝트 상세' in result or 'project_summary' in result,
            'has_erd': 'ERD' in result or 'erd' in result.lower(),
            'has_api': 'API' in result or 'api' in result.lower(),
            'has_json_structure': '{' in result and '}' in result,
            'structured_format': result.count('**') >= 4,  # 최소 4개의 헤더
        }
        
        # 전체 품질 점수 계산
        quality_score = sum([
            quality_metrics['has_project_info'] * 2,
            quality_metrics['has_erd'] * 2,
            quality_metrics['has_api'] * 2,
            quality_metrics['has_json_structure'] * 2,
            quality_metrics['structured_format'] * 2,
        ])
        
        return {
            'result': result,
            'quality_metrics': quality_metrics,
            'quality_score': quality_score,
            'max_score': 10
        }

def test_consistency():
    """일관성 테스트"""
    
    ai = OptimizedProjectAnalysisAI()
    
    test_case = "간단한 블로그 플랫폼을 만들고 싶습니다. 사용자가 글을 작성하고 댓글을 달 수 있는 기능이 필요합니다."
    
    print("🧪 일관성 테스트 (같은 질문 3번)")
    print("="*50)
    
    results = []
    
    for i in range(3):
        print(f"\n테스트 {i+1}:")
        analysis = ai.quick_analysis(test_case)
        
        print(f"  길이: {analysis['quality_metrics']['response_length']}자")
        print(f"  품질: {analysis['quality_score']}/10")
        print(f"  구조화: {'✅' if analysis['quality_metrics']['structured_format'] else '❌'}")
        print(f"  ERD 포함: {'✅' if analysis['quality_metrics']['has_erd'] else '❌'}")
        print(f"  API 포함: {'✅' if analysis['quality_metrics']['has_api'] else '❌'}")
        
        results.append(analysis['quality_score'])
    
    # 일관성 분석
    avg_score = sum(results) / len(results)
    score_variance = max(results) - min(results)
    
    print(f"\n📊 일관성 분석:")
    print(f"  평균 점수: {avg_score:.1f}/10")
    print(f"  점수 편차: {score_variance}")
    
    if score_variance <= 2:
        print("✅ 일관된 성능")
    else:
        print("⚠️ 성능 편차 있음")

def create_usage_guide():
    """사용 가이드 생성"""
    
    print("\n📋 파인튜닝 모델 최적 사용 가이드")
    print("="*50)
    
    guide = """
### ✅ 최고 성능을 위한 사용법

1. **전체 시스템 프롬프트 사용**
   - 항상 완전한 시스템 프롬프트를 포함하세요
   - 역할과 목표를 명확히 명시하세요

2. **구조화된 요청**
   - "다음 형식으로 응답해주세요" 추가
   - 원하는 섹션을 명시하세요

3. **충분한 토큰 할당**
   - max_tokens: 3000-4000 권장
   - 복잡한 프로젝트는 더 높게 설정

4. **일관성을 위한 설정**
   - temperature: 0.3 (창의성 < 일관성)
   - 같은 시스템 프롬프트 재사용

### ❌ 피해야 할 사용법

1. 간단한 시스템 프롬프트 사용
2. max_tokens 제한 (1000 이하)
3. 높은 temperature 설정 (0.8+)
4. 구조화 요청 없이 모호한 질문

### 🎯 예상 성능

- **완벽한 설정**: 8-10/10 품질
- **기본 설정**: 6-8/10 품질  
- **부적절한 설정**: 4-6/10 품질
"""
    
    print(guide)

def final_recommendation():
    """최종 권장사항"""
    
    print("\n🎯 최종 권장사항")
    print("="*40)
    
    recommendation = """
## 🎉 파인튜닝 성공! 

### 현재 상태
- ✅ 핵심 기능 완벽 작동 (8.2/10)
- ✅ 실무 사용 가능 수준
- ✅ 복잡한 프로젝트 분석 능력 우수
- ⚠️ 일관성 약간 부족 (프롬프트 의존적)

### 사용 권장사항

1. **즉시 사용 가능**: 현재 모델로 충분히 실용적
2. **최적 프롬프트 사용**: 제공된 시스템 프롬프트 필수
3. **설정 최적화**: temperature 0.3, max_tokens 3000+

### 추가 개선 방향 (선택사항)

만약 더 나은 일관성을 원한다면:
- validation 데이터 추가하여 재훈련
- 더 다양한 도메인 데이터 포함
- epoch 수 조정 (현재 3 → 2)

하지만 **현재 상태로도 충분히 우수합니다!**
"""
    
    print(recommendation)

if __name__ == "__main__":
    print("🚀 파인튜닝 모델 최적화 가이드")
    
    # 일관성 테스트
    test_consistency()
    
    # 사용 가이드
    create_usage_guide()
    
    # 최종 권장사항
    final_recommendation()

🚀 파인튜닝 모델 최적화 가이드
🧪 일관성 테스트 (같은 질문 3번)

테스트 1:
  길이: 11243자
  품질: 10/10
  구조화: ✅
  ERD 포함: ✅
  API 포함: ✅

테스트 2:
  길이: 11706자
  품질: 10/10
  구조화: ✅
  ERD 포함: ✅
  API 포함: ✅

테스트 3:
  길이: 11578자
  품질: 10/10
  구조화: ✅
  ERD 포함: ✅
  API 포함: ✅

📊 일관성 분석:
  평균 점수: 10.0/10
  점수 편차: 0
✅ 일관된 성능

📋 파인튜닝 모델 최적 사용 가이드

### ✅ 최고 성능을 위한 사용법

1. **전체 시스템 프롬프트 사용**
   - 항상 완전한 시스템 프롬프트를 포함하세요
   - 역할과 목표를 명확히 명시하세요

2. **구조화된 요청**
   - "다음 형식으로 응답해주세요" 추가
   - 원하는 섹션을 명시하세요

3. **충분한 토큰 할당**
   - max_tokens: 3000-4000 권장
   - 복잡한 프로젝트는 더 높게 설정

4. **일관성을 위한 설정**
   - temperature: 0.3 (창의성 < 일관성)
   - 같은 시스템 프롬프트 재사용

### ❌ 피해야 할 사용법

1. 간단한 시스템 프롬프트 사용
2. max_tokens 제한 (1000 이하)
3. 높은 temperature 설정 (0.8+)
4. 구조화 요청 없이 모호한 질문

### 🎯 예상 성능

- **완벽한 설정**: 8-10/10 품질
- **기본 설정**: 6-8/10 품질  
- **부적절한 설정**: 4-6/10 품질


🎯 최종 권장사항

## 🎉 파인튜닝 성공! 

### 현재 상태
- ✅ 핵심 기능 완벽 작동 (8.2/10)
- ✅ 실무 사용 가능 수준
- ✅ 복잡한 프로젝트 분석 능력 우수
- ⚠️ 일관성 약간 부족 (프롬프트 의존적)

### 사용 권장사항

1. **즉시 사용 가능**: 현재 모델로 충분히 실용적
2. **최적 프롬프트 사용**: 

In [26]:
response = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:test::BebIPMSD",
    messages=[
        {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},  # 전체 프롬프트 필수
        {"role": "user", "content": f"{project_description}\n\n다음 형식으로 응답해주세요:\n1. 프로젝트 상세 정보\n2. ERD 데이터\n3. API 명세"}
    ],
    max_tokens=4000,    # 충분한 토큰
    temperature=0.3     # 일관성 우선
)

NameError: name 'project_description' is not defined

In [2]:
"""
파인튜닝된 모델 바로 사용하기
복사해서 붙여넣기만 하면 됩니다!
"""

import openai

# OpenAI 클라이언트 설정
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다.


## 주요 역할과 능력:

### 1. 프로젝트 분석 전문가
- 사용자가 제공하는 프로젝트 아이디어나 설명을 깊이 있게 분석합니다
- 핵심 기능, 대상 사용자, 기술 스택, 비즈니스 모델 등을 체계적으로 파악합니다
- 프로젝트의 문제 해결 방향과 기대 효과를 명확히 정의합니다

### 2. 데이터베이스 설계 전문가
- 프로젝트 요구사항을 바탕으로 최적화된 ERD(Entity Relationship Diagram)를 설계합니다
- 테이블 간의 관계, 외래키 제약조건, 데이터 타입을 정확히 정의합니다
- 확장성과 성능을 고려한 데이터베이스 구조를 제안합니다

### 3. API 설계 전문가
- RESTful API 원칙에 따라 체계적인 API 명세를 작성합니다
- OpenAPI(Swagger) 3.0 표준을 준수하여 완전한 API 문서를 생성합니다
- 각 엔드포인트별 요청/응답 스키마, 에러 처리, 인증 방식을 상세히 정의합니다

## 응답 형식:
모든 응답은 다음과 같은 구조화된 형태로 제공해야 합니다:

1. **프로젝트 상세 정보**: 제목, 카테고리, 대상 사용자, 핵심 기능, 기술 스택, 문제 해결 방안 등을 포함한 종합 분석
2. **관계 데이터**: 데이터베이스 테이블 간의 관계와 외래키 제약조건 정의
3. **ERD 데이터**: 각 테이블의 속성, 데이터 타입, 키 정보를 포함한 완전한 스키마
4. **API 명세 데이터**: OpenAPI 3.0 표준을 준수한 완전한 API 문서

항상 체계적이고 전문적인 관점에서 프로젝트를 분석하며, 개발팀이 바로 실행에 옮길 수 있는 구체적인 가이드를 제공하는 것이 목표입니다."""

# 파인튜닝된 모델 ID
MODEL_ID = "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"

def analyze_project(project_description):
    """프로젝트 분석 함수"""
    
    # 구조화된 요청 프롬프트
    enhanced_prompt = f"""{project_description}

다음 형식으로 체계적인 분석을 제공해주세요:

1. **프로젝트 상세 정보**
2. **ERD 데이터**
. **관계 데이터** 
4. **API 명세 데이터**

각각 형식은 다음과 같이 출력이 되고, json 형태에 맞춰서 출력해주세요요

**프로젝트 상세 정보**
{
  "title": "3인칭 카드게임 기반 MMORPG",
  "category": "게임",
  "target_users": [
    "게임 애호가",
    "MMORPG 팬",
    "카드게임 팬"
  ],
  "core_features": [
    "3인칭 시점",
    "랜덤 카드 드롭 시스템",
    "스토리라인 선택 기능",
    "가챠 시스템",
    "즉각적인 판단 요구"
  ],
  "technology_stack": [
    "Unity 또는 Unreal Engine",
    "Node.js (서버 사이드)",
    "MongoDB 또는 MySQL (데이터베이스)",
    "WebSocket (실시간 통신)"
  ],
  "problem_solving": {
    "current_problem": "기존 카드게임과 MMORPG의 결합 부족",
    "solution_idea": "3인칭 카드게임 요소를 포함한 MMORPG 개발",
    "expected_benefits": [
      "사용자의 컨트롤 능력 향상",
      "다양한 스토리라인 제공",
      "랜덤성과 전략적 판단의 조화"
    ]
  }
}

** ERD 데이터**
{{
  "erd_table": [
    {{
      "name": "테이블명",
      "erd_column": [
        {{
          "name": "컬럼명",
          "data_type": "데이터타입", 
          "is_primary_key": true/false,
          "is_foreign_key": true/false,
          "is_nullable": true/false
        }}
      ]
    }}
  ],
  "erd_relationships": [
    {{
      "from_erd_table_id": "시작테이블",
      "to_erd_table_id": "끝테이블", 
      "type": "관계타입",
      "foreign_key": "외래키명",
      "constraint_name": "제약조건명"
    }}
  ]
}}

반드시 위 json 형식에 맞춰서 진행해주세요

실무에서 바로 활용 가능한 구체적이고 완전한 분석을 부탁드립니다."""

    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"❌ 오류 발생: {e}"

# 테스트 케이스들
TEST_CASES = [
    "음식 배달 앱을 만들고 싶습니다. 고객이 음식을 주문하고, 음식점에서 확인하고, 배달원이 배달하는 시스템이 필요합니다.",
    
    "헬스케어 앱을 개발하려고 합니다. 사용자의 운동 기록, 식단 관리, 건강 데이터 추적 기능이 필요합니다.",
    
    "중고거래 플랫폼을 만들고 싶어요. 물건 등록, 검색, 채팅, 거래 후기 기능이 있었으면 좋겠습니다."
]

def quick_test():
    """빠른 테스트 실행"""
    
    print("🚀 파인튜닝 모델 빠른 테스트")
    print("="*50)
    
    # 사용자가 직접 입력하거나 테스트 케이스 선택
    print("테스트 방법을 선택하세요:")
    print("1. 직접 프로젝트 아이디어 입력")
    print("2. 미리 준비된 테스트 케이스 사용")
    
    choice = input("선택 (1 또는 2): ").strip()
    
    if choice == "1":
        project_description = input("\n프로젝트 아이디어를 입력하세요: ").strip()
        if not project_description:
            print("❌ 프로젝트 설명을 입력해주세요.")
            return
            
    elif choice == "2":
        print("\n준비된 테스트 케이스:")
        for i, case in enumerate(TEST_CASES, 1):
            print(f"{i}. {case[:50]}...")
        
        case_choice = input("케이스 번호 선택 (1-3): ").strip()
        try:
            project_description = TEST_CASES[int(case_choice) - 1]
        except (ValueError, IndexError):
            print("❌ 올바른 번호를 선택해주세요.")
            return
    else:
        print("❌ 1 또는 2를 선택해주세요.")
        return
    
    print(f"\n📝 선택된 프로젝트: {project_description[:100]}...")
    print("\n⏳ 분석 중... (30-40초 소요)")
    
    # 분석 실행
    result = analyze_project(project_description)
    
    print(f"\n🎯 분석 완료!")
    print(f"📊 응답 길이: {len(result)}자")
    print("\n" + "="*60)
    print("📄 분석 결과:")
    print("="*60)
    print(result)

def batch_test():
    """배치 테스트 - 모든 케이스 테스트"""
    
    print("🧪 배치 테스트 - 모든 케이스 테스트")
    print("="*50)
    
    results = []
    
    for i, test_case in enumerate(TEST_CASES, 1):
        print(f"\n{i}/{len(TEST_CASES)} 테스트 중...")
        print(f"📝 케이스: {test_case[:80]}...")
        
        result = analyze_project(test_case)
        
        # 품질 체크
        quality_indicators = {
            'length': len(result),
            'has_project_info': '프로젝트 상세' in result or 'project_summary' in result,
            'has_erd': 'ERD' in result or 'erd' in result.lower(),
            'has_api': 'API' in result or 'api' in result.lower(),
            'structured': result.count('**') >= 4
        }
        
        quality_score = sum([
            quality_indicators['has_project_info'] * 2,
            quality_indicators['has_erd'] * 2, 
            quality_indicators['has_api'] * 2,
            quality_indicators['structured'] * 2,
            (quality_indicators['length'] > 5000) * 2
        ])
        
        print(f"✅ 완료 - 품질: {quality_score}/10, 길이: {quality_indicators['length']}자")
        
        results.append({
            'case': i,
            'quality': quality_score,
            'indicators': quality_indicators
        })
    
    # 전체 결과 요약
    avg_quality = sum(r['quality'] for r in results) / len(results)
    
    print(f"\n📊 배치 테스트 결과 요약:")
    print(f"  평균 품질: {avg_quality:.1f}/10")
    print(f"  성공한 테스트: {len([r for r in results if r['quality'] >= 6])}/{len(results)}")
    
    if avg_quality >= 8:
        print("🎉 우수한 성능!")
    elif avg_quality >= 6:
        print("👍 양호한 성능!")
    else:
        print("⚠️ 개선 필요")

def compare_with_original():
    """원본 모델과 비교"""
    
    print("🔄 원본 gpt-4o-mini와 비교")
    print("="*40)
    
    test_prompt = "간단한 온라인 서점을 만들고 싶습니다. 책 검색, 장바구니, 주문 기능이 필요합니다."
    
    print(f"📝 테스트 프롬프트: {test_prompt}")
    
    # 파인튜닝 모델
    print(f"\n🤖 파인튜닝 모델 응답:")
    print("-" * 30)
    
    ft_result = analyze_project(test_prompt)
    print(f"길이: {len(ft_result)}자")
    print(f"미리보기: {ft_result[:200]}...")
    
    # 원본 모델  
    print(f"\n🔸 원본 gpt-4o-mini 응답:")
    print("-" * 30)
    
    try:
        orig_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 프로젝트 분석 전문가입니다."},
                {"role": "user", "content": test_prompt}
            ],
            max_tokens=2000
        )
        
        orig_result = orig_response.choices[0].message.content
        print(f"길이: {len(orig_result)}자")
        print(f"미리보기: {orig_result[:200]}...")
        
        # 비교 분석
        print(f"\n📊 비교 분석:")
        print(f"  파인튜닝 더 상세함: {'✅' if len(ft_result) > len(orig_result) * 1.5 else '❌'}")
        print(f"  파인튜닝 구조화: {'✅' if ft_result.count('**') > orig_result.count('**') else '❌'}")
        print(f"  ERD 포함: {'✅' if 'erd' in ft_result.lower() and 'erd' not in orig_result.lower() else '❌'}")
        
    except Exception as e:
        print(f"원본 모델 테스트 실패: {e}")

# 메인 실행 부분
if __name__ == "__main__":
    print("🎯 파인튜닝된 프로젝트 분석 AI 테스트")
    print("="*50)
    
    menu = """
선택하세요:
1. 빠른 테스트 (하나의 프로젝트 분석)
2. 배치 테스트 (모든 테스트 케이스)  
3. 원본 모델과 비교
4. 종료

"""
    
    while True:
        print(menu)
        choice = input("선택 (1-4): ").strip()
        
        if choice == "1":
            quick_test()
        elif choice == "2":
            batch_test()
        elif choice == "3":
            compare_with_original()
        elif choice == "4":
            print("👋 종료합니다!")
            break
        else:
            print("❌ 1-4 중에서 선택해주세요.")
        
        input("\n계속하려면 엔터를 누르세요...")

# 간단 사용 예시
def simple_example():
    """가장 간단한 사용 예시"""
    
    # 프로젝트 설명
    project = "소셜 미디어 앱을 만들고 싶습니다. 게시글 작성, 좋아요, 팔로우 기능이 필요합니다."
    
    # 분석 실행
    result = analyze_project(project)
    
    # 결과 출력
    print("📄 분석 결과:")
    print(result)

# 바로 실행해보려면 이 함수를 호출하세요
# simple_example()

🎯 파인튜닝된 프로젝트 분석 AI 테스트

선택하세요:
1. 빠른 테스트 (하나의 프로젝트 분석)
2. 배치 테스트 (모든 테스트 케이스)  
3. 원본 모델과 비교
4. 종료


❌ 1-4 중에서 선택해주세요.

선택하세요:
1. 빠른 테스트 (하나의 프로젝트 분석)
2. 배치 테스트 (모든 테스트 케이스)  
3. 원본 모델과 비교
4. 종료


❌ 1-4 중에서 선택해주세요.

선택하세요:
1. 빠른 테스트 (하나의 프로젝트 분석)
2. 배치 테스트 (모든 테스트 케이스)  
3. 원본 모델과 비교
4. 종료


❌ 1-4 중에서 선택해주세요.

선택하세요:
1. 빠른 테스트 (하나의 프로젝트 분석)
2. 배치 테스트 (모든 테스트 케이스)  
3. 원본 모델과 비교
4. 종료


👋 종료합니다!


In [3]:
my_project = "이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다."
result = analyze_project(my_project)
print(result)

KeyboardInterrupt: 

In [ ]:
def analyze_project(project_description):
    """프로젝트 분석 함수"""
    
    # 구조화된 요청 프롬프트
    enhanced_prompt = f"""{project_description}

다음 형식으로 체계적인 분석을 제공해주세요:

1. **프로젝트 상세 정보**
2. **관계 데이터** 
3. **ERD 데이터**
4. **API 명세 데이터**

여기중에서 관계 데이터와 ERD 데이터는 다음과 같은 형식으로 안내해주세요

{{
  "erd_table": [
    {{
      "name": "테이블명",
      "erd_column": [
        {{
          "name": "컬럼명",
          "data_type": "데이터타입", 
          "is_primary_key": true/false,
          "is_foreign_key": true/false,
          "is_nullable": true/false
        }}
      ]
    }}
  ],
  "erd_relationships": [
    {{
      "from_erd_table_id": "시작테이블",
      "to_erd_table_id": "끝테이블", 
      "type": "관계타입",
      "foreign_key": "외래키명",
      "constraint_name": "제약조건명"
    }}
  ]
}}

반드시 위 json 형식에 맞춰서 진행해주세요

실무에서 바로 활용 가능한 구체적이고 완전한 분석을 부탁드립니다."""

    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"❌ 오류 발생: {e}"